In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 10 11:48:11 2023

@author: hrajagopalan3
"""
import cantera as ct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import integrate
import multiprocessing
import time


In [ ]:

def make_callback(flame):
    """
    Create and return a callback function that you will attach to
    a flame solver. The reason we define a function to make the callback function,
    instead of just defining the callback function, is so that it can store
    a pair of lists that persist between function calls, to store the
    values of grid size and flame speed.

    This factory returns the callback function, and the two lists:
    (callback, speeds, grids)
    """
    speeds = []
    grids = []

    def callback(_):
        speed = flame.velocity[0]
        grid = len(flame.grid)
        speeds.append(speed)
        grids.append(grid)
        print(f"Iteration {len(grids)}")
        print(f"Current flame speed is is {speed * 100:.4f} cm/s")
        if len(grids) < 5:
            return 1.0  #
        try:
            extrapolate_uncertainty(grids, speeds)
        except Exception as e:
            print("Couldn't estimate uncertainty. " + str(e))
            return 1.0  # continue anyway
        return 1.0

    return callback, speeds, grids

def extrapolate_uncertainty(grids, speeds, plot=True):
    """
    Given a list of grid sizes and a corresponding list of flame speeds,
    extrapolate and estimate the uncertainty in the final flame speed.
    Also makes a plot, unless called with `plot=False`.
    """
    grids = list(grids)
    speeds = list(speeds)

    def speed_from_grid_size(grid_size, true_speed, error):
        """
        Given a grid size (or an array or list of grid sizes)
        return a prediction (or array of predictions)
        of the computed flame speed, based on
        the parameters `true_speed` and `error`.

        It seems, from experience, that error scales roughly with
        1/grid_size, so we assume that form.
        """
        return true_speed + error / np.array(grid_size)

    # Fit the chosen form of speed_from_grid_size, to the last four
    # speed and grid size values.
    popt, pcov = scipy.optimize.curve_fit(speed_from_grid_size, grids[-4:], speeds[-4:])

    # How bad the fit was gives you some error, `percent_error_in_true_speed`.
    perr = np.sqrt(np.diag(pcov))
    true_speed_estimate = popt[0]
    percent_error_in_true_speed = perr[0] / popt[0]
    # print(
    #     f"Fitted true_speed is {popt[0] * 100:.4f} ± {perr[0] * 100:.4f} cm/s "
    #     f"({percent_error_in_true_speed:.1%})"
    # )

    # How far your extrapolated infinite grid value is from your extrapolated
    # (or interpolated) final grid value, gives you some other error, `estimated_percent_error`
    estimated_percent_error = (
        speed_from_grid_size(grids[-1], *popt) - true_speed_estimate
    ) / true_speed_estimate
    print(f"Estimated error in final calculation {estimated_percent_error:.1%}")

    # The total estimated error is the sum of these two errors.
    total_percent_error_estimate = abs(percent_error_in_true_speed) + abs(
        estimated_percent_error
    )
    print(f"Estimated total error {total_percent_error_estimate:.1%}")

    if plot:
        plt.semilogx(grids, speeds, "o-")
        plt.ylim(
            min(speeds[-5:] + [true_speed_estimate - perr[0]]) * 0.95,
            max(speeds[-5:] + [true_speed_estimate + perr[0]]) * 1.05,
        )
        plt.plot(grids[-4:], speeds[-4:], "or")
        extrapolated_grids = grids + [grids[-1] * i for i in range(2, 8)]
        plt.plot(
            extrapolated_grids, speed_from_grid_size(extrapolated_grids, *popt), ":r"
        )
        plt.xlim(*plt.xlim())
        plt.hlines(true_speed_estimate, *plt.xlim(), colors="r", linestyles="dashed")
        plt.hlines(
            true_speed_estimate + perr[0],
            *plt.xlim(),
            colors="r",
            linestyles="dashed",
            alpha=0.3,
        )
        plt.hlines(
            true_speed_estimate - perr[0],
            *plt.xlim(),
            colors="r",
            linestyles="dashed",
            alpha=0.3,
        )
        plt.fill_between(
            plt.xlim(),
            true_speed_estimate - perr[0],
            true_speed_estimate + perr[0],
            facecolor="red",
            alpha=0.1,
        )

        above = popt[1] / abs(
            popt[1]
        )  # will be +1 if approach from above or -1 if approach from below

        plt.annotate(
            "",
            xy=(grids[-1], true_speed_estimate),
            xycoords="data",
            xytext=(grids[-1], speed_from_grid_size(grids[-1], *popt)),
            textcoords="data",
            arrowprops=dict(
                arrowstyle="|-|, widthA=0.5, widthB=0.5",
                linewidth=1,
                connectionstyle="arc3",
                color="black",
                shrinkA=0,
                shrinkB=0,
            ),
        )

        plt.annotate(
            f"{abs(estimated_percent_error):.1%}",
            xy=(grids[-1], speed_from_grid_size(grids[-1], *popt)),
            xycoords="data",
            xytext=(5, 15 * above),
            va="center",
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3"),
        )

        plt.annotate(
            "",
            xy=(grids[-1] * 4, true_speed_estimate - (above * perr[0])),
            xycoords="data",
            xytext=(grids[-1] * 4, true_speed_estimate),
            textcoords="data",
            arrowprops=dict(
                arrowstyle="|-|, widthA=0.5, widthB=0.5",
                linewidth=1,
                connectionstyle="arc3",
                color="black",
                shrinkA=0,
                shrinkB=0,
            ),
        )
        plt.annotate(
            f"{abs(percent_error_in_true_speed):.1%}",
            xy=(grids[-1] * 4, true_speed_estimate - (above * perr[0])),
            xycoords="data",
            xytext=(5, -15 * above),
            va="center",
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3"),
        )

        plt.ylabel("Flame speed (m/s)")
        plt.xlabel("Grid size")
        plt.show()

    return true_speed_estimate, total_percent_error_estimate


def flame(P,T, phi, fuel_mix, oxidizer_mix, mechanism):
    start = time.time()
    # Initialize gas properties and chem mech
    gas = ct.Solution(mechanism)
    gas.TP = T,P
    gas.set_equivalence_ratio(phi, fuel=fuel_mix, oxidizer=oxidizer_mix)

    f = ct.FreeFlame(gas)
    f.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)

    # Solve with mixture-averaged transport model
    f.transport_model = 'Multi'
    loglevel = 0
    f.solve(loglevel=loglevel, auto=True)
    x         = f.grid
    sl        = f.velocity[0] # Flame speed: Index is based on grid points
    Tad       = f.T[-1] # Product gas temperature: : Index is based on grid points
    cp        = np.average(f.cp_mass[0]) # Reactant Cp: Index is based on grid points
    rho_b     = f.density[-1] # Product gas density: : Index is based on grid points
    rho_u     = f.density[0] # Reactant gas density: : Index is based on grid points
    lambd     = f.thermal_conductivity[0] # Thermal conductivity: Index is based on grid points
    alpha     = lambd/(rho_u*cp) # thermal diffusivity: from rxn zone to reactants
    D         = f.mix_diff_coeffs[int(gas.species_index('H2'))][0] # mass diffusivity: of H2
    #Dav       = np.average(f.mix_diff_coeffs[:][0]) # average mass diffusivity of product gas species
    mu        = f.viscosity[0] # reactant viscosity: Index is based on grid points
    nu        = mu/rho_u
    ft_t      = (Tad-f.T[0])/(np.max(np.diff(f.T)/np.diff(f.grid)))# Thermal flame thickness
    ft_d      = alpha/sl #flame thickness (diffusion based definition)
    Pr        = (nu/alpha) # Prandtl number
    Le        = alpha/D # H2 Lewis number
    Sc        = (nu/D) # Schmidt number (based on D of H2)
    NO_arr    = f.X[int(gas.species_index('NO'))]
    CO_arr    = f.Y[int(gas.species_index('CO'))]
    CO2_arr    = f.Y[int(gas.species_index('CO2'))]
    H2O_arr    = f.Y[int(gas.species_index('H2O'))]


    end=time.time()
    print(f"Completed:|Time taken:{end-start}")
    return {'Distance [m]':[x],
        'rhou [kg/m3]':rho_u, 
            'rhob [kg/m3]':rho_b,
            'alpha [m2/s]':alpha,
            'nu [m2/s]':nu,
            'D [m2/s]':D,
            'Le H2':Le,
            'Pr':Pr,
            'Sc':Sc,
            'SL [m/s]':sl,
            'ft_t [m]':ft_t,
            'ft_d [m]':ft_d,
            'ft_d [m]':ft_d*39.37,
            'Tad [K]':Tad,
            'Tad [F]':((Tad-273.15)*1.8)+32,
            'T [K]': [f.T],
            'NO [ppm]': [NO_arr],
            'CO [ppm]':[CO_arr],
            'CO2 [ppm]':[CO2_arr],
            'H2O [ppm]':[H2O_arr]}


In [ ]:
# ------------------------------------------------- Main function -------------
# df =pd.read_excel(r"SampleData.xlsx",sheet_name = 'GTTFS')
# df=df.dropna(how='all')
# df=df.reset_index()

# BC for T130 

Tall=[724.8]
Pall=[18.5]
# phiall=[1]
phiall =(np.linspace(0.4,3,100))
alphas = [0,25,50,60,75,80,90,100]
all_results=[] 
print(phiall)



In [ ]:


if __name__ == '__main__':

   for j in range(len(alphas)):
        H2_fraction=alphas[j]/100
        for i in range(len(phiall)):
                T=Tall[0]
                P = Pall[0]* ct.one_atm  # Convert atm to Pa
                phi=phiall[i]
                fuel_mix=f"CH4:{1 - H2_fraction}, H2:{H2_fraction}"
                oxidizer_mix={"O2": 1.0, "N2": 3.76}
                mechanism= r"C:\Users\d69242\OneDrive - Caterpillar\ChemicalKinetics\Mechanisms\CSU45_S45_R245_TNO\CSU45_S45_R245_TNO\CSU45_S45_R425_TNO.yaml"
                inputs=[P,T, phi, fuel_mix, oxidizer_mix,mechanism]
                results = flame(P, T, phi, fuel_mix, oxidizer_mix, mechanism)  # flame() should return a dict
                tagged_results = {
                "phi": phi,
                "P [psi]": P/6894.75,
                "T [F]": ((T-273.15)*1.8)+32,
                "H2_fraction": H2_fraction,
                # you could also rebuild fuel_mix if you want:
                "fuel_mix": f"CH4:{1 - H2_fraction}, H2:{H2_fraction}",
                **results
                }

                all_results.append(tagged_results)  # append the dict to list


In [ ]:

    # Convert the list of dictionaries to a DataFrame
df = pd.read_excel("CH4FlameCalcsGRI30.xlsx")
#df["Phi"]=phiall
plt.figure(dpi=500, figsize=(5,4))

for idx,row in df.iterrows():
    x = row["Distance (m)"][0]
    y = row["T(K)"][0]
    #PV = row["CO"][0] +  row["CO2_arr"][0]
    plt.plot(x,y,color="blue")


plt.xlabel(r"x [m]")
plt.ylabel(r"Temperature [K]")




In [ ]:


df = pd.DataFrame(all_results)
df.to_csv("T130FlamePropsCSU45.csv", index=False)



In [ ]:
#df=pd.read_excel(r"C:\Users\d69242\OneDrive - Caterpillar\JupyterNB\FlamePropsCSU45T130.xlsx")
df=pd.DataFrame(all_results)

print(df.columns)


In [ ]:


plt.figure(dpi=500,figsize=(5,4))
plt.scatter(phiall,df["SL (m/s)"])
plt.xlabel("Phi")
plt.ylabel("Flame speed [m/s]")
plt.savefig("PhivSL.png")


In [ ]:

print(np.unique(df["H2_fraction"]))